# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import json
from loguru import logger
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time

import mlflow

import sys
sys.path.insert(0, '..')

In [3]:
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [5]:
TESTING = False
DEBUG = True
OBSERVABILITY = True
LOG_TO_MLFLOW = True

In [6]:
if OBSERVABILITY:
    import phoenix as px
    px.launch_app()
    import llama_index.core
    llama_index.core.set_global_handler("arize_phoenix")

I0000 00:00:1721823468.132393  582382 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [7]:
import logging
import sys

if DEBUG:
    logging.getLogger('llama_index').addHandler(logging.StreamHandler(stream=sys.stdout))
    logging.getLogger('llama_index').setLevel(logging.DEBUG)

In [8]:
RUN_NAME = "exp_003_reranker_flag_embedding_bge_large"
if LOG_TO_MLFLOW:
    RUN_DESCRIPTION = """
# Making the RAG outputs referenced sources

## Changelog
### Compares to exp_002
- Update source data to have URLs as well
- First attempt to add reranker
"""
    mlflow.set_experiment("Chain Frost - LlamaIndex Blog QnA Chatbot")
    mlflow.start_run(run_name=RUN_NAME, description=RUN_DESCRIPTION)
    mlflow.log_param("TESTING", TESTING)

In [9]:
NOTEBOOK_CACHE_DP = f'data/001/{RUN_NAME}'
os.makedirs(NOTEBOOK_CACHE_DP, exist_ok=True)

if LOG_TO_MLFLOW:
    mlflow.log_param("NOTEBOOK_CACHE_DP", NOTEBOOK_CACHE_DP)

# Load data

In [10]:
DATA_FP = '../crawl_llamaindex_blog/data/blogs-v2.json'
with open(DATA_FP, 'r') as f:
    data = json.load(f)

In [11]:
len(data)

160

In [12]:
data[:5]

[{'title': 'Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations',
  'content': "This is a guest post from Uptrain. We are excited to announce the recent integration of LlamaIndex with UpTrain - an open-source LLM evaluation framework to evaluate your RAG pipelines and experiment with different configurations. As an increasing number of companies are graduating their LLM prototypes to production-ready systems, robust evaluations provide a systematic framework to make decisions rather than going with the ‘vibes’. By combining LlamaIndex's flexibility and UpTrain's evaluation framework, developers can experiment with different configurations, fine-tuning their LLM-based applications for optimal performance. About UpTrain UpTrain  [ github  ||  website  ||  docs ] is an open-source platform to evaluate and improve LLM applications. It provides grades for 20+ preconfigured checks (covering language, code, embedding use cases), performs root cause analyses on instances of failu

# Check data

In [13]:
data[0]['content']

"This is a guest post from Uptrain. We are excited to announce the recent integration of LlamaIndex with UpTrain - an open-source LLM evaluation framework to evaluate your RAG pipelines and experiment with different configurations. As an increasing number of companies are graduating their LLM prototypes to production-ready systems, robust evaluations provide a systematic framework to make decisions rather than going with the ‘vibes’. By combining LlamaIndex's flexibility and UpTrain's evaluation framework, developers can experiment with different configurations, fine-tuning their LLM-based applications for optimal performance. About UpTrain UpTrain  [ github  ||  website  ||  docs ] is an open-source platform to evaluate and improve LLM applications. It provides grades for 20+ preconfigured checks (covering language, code, embedding use cases), performs root cause analyses on instances of failure cases and provides guidance for resolving them. Key Highlights: Data Security:  As an open

# Prepare documents

In [14]:
input_data = data
if TESTING:
    input_data = data[:2]
logger.info(f"{len(input_data)=}")
if LOG_TO_MLFLOW:
    mlflow.log_param("len_input_data", len(input_data))

2024-07-24 19:17:51.576 | INFO     | __main__:<module>:4 - len(input_data)=160


In [15]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

documents = []
for record in input_data:
    title = record['title']
    metadata = {
        'title': title,
        'author': record['author'],
        'date': record['date'],
        'tags': ', '.join(record['tags']),
        'url': record['url']
    }
    text = f"{title}\n{record['content']}"
    doc = Document(text=text, metadata=metadata)
    documents.append(doc)

In [16]:
documents[0]

Document(id_='dd4374f1-3b6e-4dce-b664-ac47324a99b8', embedding=None, metadata={'title': 'Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations', 'author': 'Uptrain', 'date': 'Mar 19, 2024', 'tags': 'AI, Evaluation, Rag', 'url': 'https://www.llamaindex.ai/blog/supercharge-your-llamaindex-rag-pipeline-with-uptrain-evaluations'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations\nThis is a guest post from Uptrain. We are excited to announce the recent integration of LlamaIndex with UpTrain - an open-source LLM evaluation framework to evaluate your RAG pipelines and experiment with different configurations. As an increasing number of companies are graduating their LLM prototypes to production-ready systems, robust evaluations provide a systematic framework to make decisions rather than going with the ‘vibes’. By combining LlamaIndex's flexibility and UpTrain's evaluation fram

In [17]:
documents[1].metadata

{'title': 'LlamaIndex Newsletter 2024-04-02',
 'author': 'LlamaIndex',
 'date': 'Apr 2, 2024',
 'tags': 'LLM',
 'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-04-02'}

In [18]:
if LOG_TO_MLFLOW:
    mlflow.log_param("len_documents", len(documents))

## Setting LLM

In [19]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, ServiceContext

In [20]:
# LLM_OPTION = 'openai'
# LLM_OPTION = 'ollama'
LLM_OPTION = 'togetherai'

# LLM_MODEL_NAME = 'llama3'
# LLM_MODEL_NAME = 'gpt-3.5-turbo'
LLM_MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct-Lite'

# EMBED_OPTION = 'openai'
# EMBED_OPTION = 'togetherai'
# EMBED_OPTION = 'ollama'
EMBED_OPTION = 'huggingface'

# EMBED_MODEL_NAME = 'llama3'
# EMBED_MODEL_NAME = 'togethercomputer/m2-bert-80M-2k-retrieval'
EMBED_MODEL_NAME = "BAAI/bge-large-en-v1.5"

if LOG_TO_MLFLOW:
    mlflow.log_param("LLM_OPTION", LLM_OPTION)
    mlflow.log_param("LLM_MODEL_NAME", LLM_MODEL_NAME)
    mlflow.log_param("EMBED_OPTION", EMBED_OPTION)
    mlflow.log_param("EMBED_MODEL_NAME", EMBED_MODEL_NAME)

In [21]:
# LLM options
if LLM_OPTION == 'ollama':
    LLM_SERVER_HOST = '192.168.100.14'
    LLM_SERVER_PORT = 11434
    base_url = f'http://{LLM_SERVER_HOST}:{LLM_SERVER_PORT}'
    llm = Ollama(base_url=base_url, model=LLM_MODEL_NAME, request_timeout=60.0)
    !ping -c 1 $LLM_SERVER_HOST
elif LLM_OPTION == 'openai':
    from llama_index.llms.openai import OpenAI
    llm = OpenAI(model=LLM_MODEL_NAME)
elif LLM_OPTION == 'togetherai':
    from llama_index.llms.together import TogetherLLM
    llm = TogetherLLM(model=LLM_MODEL_NAME)

# Embed options
if EMBED_OPTION == 'huggingface':
    from llama_index.embeddings.huggingface import HuggingFaceEmbedding
    embed_model = HuggingFaceEmbedding(
        model_name=EMBED_MODEL_NAME
    )
elif EMBED_OPTION == 'openai':
    from llama_index.embeddings.openai import OpenAIEmbedding
    embed_model = OpenAIEmbedding()
elif EMBED_OPTION == 'togetherai':
    from llama_index.embeddings.together import TogetherEmbedding
    embed_model = TogetherEmbedding(EMBED_MODEL_NAME)
elif EMBED_OPTION == 'ollama':
    from llama_index.embeddings.ollama import OllamaEmbedding
    embed_model = OllamaEmbedding(
        model_name=EMBED_MODEL_NAME,
        base_url=base_url,
        ollama_additional_kwargs={"mirostat": 0},
    )

logger.info(f"LLM:\n{repr(llm)}")
logger.info(f"Embed model:\n{repr(embed_model)}")

2024-07-24 19:18:08.255 | INFO     | __main__:<module>:35 - LLM:
TogetherLLM(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x731e21766210>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x731e32b4cd60>, completion_to_prompt=<function default_completion_to_prompt at 0x731e32bb3100>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='meta-llama/Meta-Llama-3-8B-Instruct-Lite', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='3cf613093b6eb9b479c341126dc8d3761c67f9340d0a4a8e1fdc62ed41b58126', api_base='https://api.together.xyz/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=False, tokenizer=None)
2024-07-24 19:18:08.256 | INFO     | __main__:<module>:36 - Embed model:
HuggingFaceEmbedding(model_name='BAAI/bge-large-e

In [22]:
embed_model_dim = len(embed_model.get_text_embedding('sample text to find embedding dimensions'))
Settings.embed_model = embed_model
Settings.llm = llm

logger.info(f"{embed_model_dim=}")

if LOG_TO_MLFLOW:
    mlflow.log_param("embedding_model_dim", embed_model_dim)
    mlflow.log_param("LLM_MODEL", repr(llm))
    mlflow.log_param("EMBEDDING_MODEL", repr(embed_model))

2024-07-24 19:18:08.824 | INFO     | __main__:<module>:5 - embed_model_dim=1024


# Index embeddings

## Qdrant as VectorStore

In [23]:
import qdrant_client
from qdrant_client.models import Distance, VectorParams
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [24]:
import string

def substitute_punctuation(text):
    # Create a translation table that maps each punctuation character to an underscore
    translator = str.maketrans(string.punctuation, '_' * len(string.punctuation))
    # Translate the text using the translation table
    return text.translate(translator)

collection_raw_name = f"{EMBED_OPTION}__{EMBED_MODEL_NAME}__{RUN_NAME}"
logger.info(f"{substitute_punctuation(collection_raw_name)=}")

2024-07-24 19:18:10.580 | INFO     | __main__:<module>:10 - substitute_punctuation(collection_raw_name)='huggingface__BAAI_bge_large_en_v1_5__exp_003_reranker_flag_embedding_bge_large'


In [25]:
RECREATE_INDEX = True

COLLECTION = substitute_punctuation(collection_raw_name)

NODES_PERSIST_FP = f'{NOTEBOOK_CACHE_DP}/nodes.pkl'
# NODES_PERSIST_FP = 'data/001/exp_001_qdrant_togetherai_llama3/nodes.pkl'

if LOG_TO_MLFLOW:
    mlflow.log_param(f"COLLECTION", COLLECTION)

In [26]:
qdrantdb = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://<host>:<port>"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="<qdrant-api-key>",
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://<host>:<port>"
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key="<qdrant-api-key>",
)
collection_exists = qdrantdb.collection_exists(COLLECTION)
if RECREATE_INDEX or not collection_exists:
    if collection_exists:
        logger.info(f"Deleting existing Qdrant collection...")
        qdrantdb.delete_collection(COLLECTION)
    if os.path.exists(NODES_PERSIST_FP):
        logger.info(f"Deleting persisted nodes object at {NODES_PERSIST_FP}...")
        os.remove(NODES_PERSIST_FP)
    logger.info(f"Creating new Qdrant collection...")
    qdrantdb.create_collection(
        COLLECTION,
        vectors_config=VectorParams(size=embed_model_dim, distance=Distance.COSINE),
    )
else:
    logger.info(f"Use existing Qdrant collection")
db_collection = qdrantdb.get_collection(COLLECTION)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=COLLECTION,
    aclient=aqdrantdb,
    prefer_grpc=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

2024-07-24 19:18:14.165 | INFO     | __main__:<module>:35 - Creating new Qdrant collection...


Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


In [27]:
CHUNKER = "SentenceSplitter"
CHUNKER_CONFIG = {
    "chunk_size": 512,
    "chunk_overlap": 10
}
if LOG_TO_MLFLOW:
    mlflow.log_param("CHUNKER", CHUNKER)
    for k, v in CHUNKER_CONFIG.items():
        mlflow.log_param(f"CHUNKER__{k}", v)

In [28]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and RECREATE_INDEX == False:
    logger.info(f"Loading index from existing DB...")
    with open(NODES_PERSIST_FP, 'rb') as f:
        nodes = pickle.load(f)
else:
    logger.info(f"Creating new DB index...")
    # Generate nodes
    # https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/
    
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache
    
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(**CHUNKER_CONFIG),
            TitleExtractor(),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # Currently setting num_workers leads to error `AttributeError: 'HuggingFaceEmbedding' object has no attribute '_model'`
    # num_workers = os.cpu_count() - 1
    # logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers)
    with open(NODES_PERSIST_FP, 'wb') as f:
        pickle.dump(nodes, f)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
t1 = time.perf_counter()

2024-07-24 19:18:24.463 | INFO     | __main__:<module>:11 - Creating new DB index...


> Adding chunk: Supercharge your LlamaIndex RAG Pipeline with U...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Supercharge your LlamaIndex RAG Pipeline with U...


> Adding chunk: In the following sections, we will illustrate h...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: In the following sections, we will illustrate h...


...

DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: This means our initial semantic search will ret...


> Adding chunk: In this dissatisfied state I went in 1988 to vi...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: In this dissatisfied state I went in 1988 to vi...


> Adding chunk: We can use re-ranking directly in RAG: query_en...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: We can use re-ranking directly in RAG: query_en...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.91it/s]
2024-07-24 19:32:27.674 | INFO     | __main__:<module>:37 - Indexing 160 into VectorStoreIndex took 843s


In [29]:
logger.info(f"Indexing {len(documents)} into VectorStoreIndex took {t1 - t0:,.0f}s")
if LOG_TO_MLFLOW:
    mlflow.log_param("len_nodes", len(nodes))

2024-07-24 19:32:27.712 | INFO     | __main__:<module>:1 - Indexing 160 into VectorStoreIndex took 843s


In [31]:
import os
import pickle
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [30]:
RECREATE_INDEX = False

COLLECTION = 'togetherai'
NOTEBOOK_CACHE_DP = 'data/001/togetherai'
NODES_PERSIST_FP = f'{NOTEBOOK_CACHE_DP}/nodes.pkl'
os.makedirs(NOTEBOOK_CACHE_DP, exist_ok=True)

In [31]:
db = chromadb.PersistentClient(path=f"{NOTEBOOK_CACHE_DP}/chroma_db")
collection_exists = COLLECTION in [c.name for c in db.list_collections()]
if RECREATE_INDEX or not collection_exists:
    logger.info(f"Creating new ChromaDB collection...")
    if collection_exists:
        logger.info(f"Deleting existing ChromaDB collection...")
        db.delete_collection(COLLECTION)
    if os.path.exists(NODES_PERSIST_FP):
        logger.info(f"Deleting persisted nodes object at {NODES_PERSIST_FP}...")
        os.remove(NODES_PERSIST_FP)
else:
    logger.info(f"Use existing ChromaDB collection")
chroma_collection = db.get_or_create_collection(COLLECTION)

2024-07-24 17:16:44.585 | INFO     | __main__:<module>:12 - Use existing ChromaDB collection


In [32]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [33]:
CHUNKER = "SentenceSplitter"
CHUNKER_CONFIG = {
    "chunk_size": 512,
    "chunk_overlap": 10
}
if LOG_TO_MLFLOW:
    mlflow.log_param("CHUNKER", CHUNKER)
    for k, v in CHUNKER_CONFIG.items():
        mlflow.log_param(f"CHUNKER__{k}", v)

In [34]:
if chroma_collection.count() > 0 and RECREATE_INDEX == False:
    logger.info(f"Loading index from existing ChromaDB...")
    with open(NODES_PERSIST_FP, 'rb') as f:
        nodes = pickle.load(f)
else:
    logger.info(f"Creating new ChromaDB index...")
    # Generate nodes
    # https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/
    
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache
    
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(**CHUNKER_CONFIG),
            TitleExtractor(),
            embedding,
        ],
        vector_store = vector_store
    )
    
    # Need to use await and arun here to run the pipeline else error
    # Ref: https://docs.llamaindex.ai/en/stable/examples/ingestion/async_ingestion_pipeline/
    # Ref: https://github.com/run-llama/llama_index/issues/13904#issuecomment-2145561710
    nodes = await pipeline.arun(documents=documents)
    with open(NODES_PERSIST_FP, 'wb') as f:
        pickle.dump(nodes, f)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

2024-07-24 17:16:44.682 | INFO     | __main__:<module>:2 - Loading index from existing ChromaDB...


#### Inspect nodes

# Query engine

In [30]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.schema import MetadataMode
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

In [31]:
from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine

In [32]:
RETRIEVAL_TOP_K = 5
RERANK_TOP_K = 2
# Need to be able to control this cutoff until specify it
RETRIEVAL_SIMILARITY_CUTOFF = None
# RETRIEVAL_SIMILARITY_CUTOFF = 0.3
# APPEND_REF_MODE = 'response_synthesizer'
APPEND_REF_MODE = 'query_engine'

if LOG_TO_MLFLOW:
    mlflow.log_param("RETRIEVAL_TOP_K", RETRIEVAL_TOP_K)
    mlflow.log_param("RERANK_TOP_K", RERANK_TOP_K)
    if RETRIEVAL_SIMILARITY_CUTOFF:
        mlflow.log_param("RETRIEVAL_SIMILARITY_CUTOFF", RETRIEVAL_SIMILARITY_CUTOFF)

In [33]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=RETRIEVAL_TOP_K,
)

node_postprocessors = []

if RETRIEVAL_SIMILARITY_CUTOFF is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=RETRIEVAL_SIMILARITY_CUTOFF))

reranker = FlagEmbeddingReranker(model="BAAI/bge-reranker-large", top_n=RERANK_TOP_K)
node_postprocessors.append(reranker)

if APPEND_REF_MODE == 'response_synthesizer':
    response_synthesizer = ManualAppendReferenceSynthesizer(verbose=0)
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=node_postprocessors,
    )
elif APPEND_REF_MODE == 'query_engine':
    response_synthesizer = get_response_synthesizer()
    query_engine = ManualAppendReferenceQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=node_postprocessors,
    )
else:
    response_synthesizer = get_response_synthesizer()
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=node_postprocessors,
    )

if LOG_TO_MLFLOW:
    mlflow.log_param("reranker", repr(reranker))
    mlflow.log_param("response_synthesizer", repr(response_synthesizer))
    mlflow.log_param("query_engine", repr(query_engine))

In [34]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [35]:
question = "How can we address points of failures in RAG pipeline?"
response = query_engine.query(question)
display_response(response, show_source=True, show_metadata=True, show_source_metadata=True)

> Top 5 nodes:
> [Node 1f8a95ea-06cc-4e05-b71b-3c3145d559d4] [Similarity score:             0.694795] " )
             return   None ,  None 
        
        embed_model = st.session_state[ 'embed_m...
> [Node ae8f1af1-9df3-48ce-8523-8050161bed54] [Similarity score:             0.692144] I have included varioud response synthesis methods supported by Llamaindex including  refine ,  t...
> [Node b9ae64b7-7f37-4523-ae89-7b2c9c6da607] [Similarity score:             0.692125] def   generate_rag_pipeline ( file, llm, embed_model, node_parser, response_mode, vector_store ):...
> [Node cb7da65e-0eae-42f1-81a6-1a50c6e0e9a7] [Similarity score:             0.691156] RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Power...
> [Node e199f568-edb2-42d7-bc2c-fbb1fef8c779] [Similarity score:             0.690548] def   select_vector_store ():
    st.header( "Choose Vector Store" )
    vector_stores = [ "Simpl...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 1f8a95ea-06cc-4e05-b71b-3c3145d559d4] [Similarity score:             0.694795] " )
             return   None ,  None 
        
        embed_model = st.session_state[ 'embed_m...
> [Node ae8f1af1-9df3-48ce-8523-8050161bed54] [Similarity score:             0.692144] I have included varioud response synthesis methods supported by Llamaindex including  refine ,  t...
> [Node b9ae64b7-7f37-4523-ae89-7b2c9c6da607] [Similarity score:             0.692125] def   generate_rag_pipeline ( file, llm, embed_model, node_parser, response_mode, vector_store ):...
> [Node cb7da65e-0eae-42f1-81a6-1a50c6e0e9a7] [Similarity score:             0.691156] RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Power...
> [Node e199f568-edb2-42d7-bc2c-fbb1fef8c779] [Similarity score:             0.690548] def   select_vector_store ():
    st.header( "Choose Vector Store" )
    vector_stores = [ "Simpl...


**`Final Response:`** To address points of failures in RAG pipeline, RAGArch provides a robust and intuitive interface for testing and debugging. The tool's live testing feature allows you to instantly test your RAG pipeline with your own data, enabling you to identify and troubleshoot any issues that may arise. Additionally, the code generation feature enables you to generate Python code for your custom RAG pipeline, making it easier to integrate and test your pipeline in your application. By leveraging LlamaIndex's powerful LLM orchestration capabilities, RAGArch provides a seamless experience and granular control over your RAG pipeline, allowing you to fine-tune and optimize your pipeline to achieve the desired results.


Sources:
- [RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Powered by LlamaIndex](https://www.llamaindex.ai/blog/ragarch-building-a-no-code-rag-pipeline-configuration-one-click-rag-code-generation-tool-powered-b6e8eeb70089)

---

**`Source Node 1/2`**

**Node ID:** cb7da65e-0eae-42f1-81a6-1a50c6e0e9a7<br>**Similarity:** -5.265542507171631<br>**Text:** RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Power...<br>**Metadata:** {'title': 'RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Powered by LlamaIndex', 'author': 'Harshad Suryawanshi', 'date': 'Feb 2, 2024', 'tags': 'Rag, No Code, Llamaindex, OpenAI, Code Generation', 'url': 'https://www.llamaindex.ai/blog/ragarch-building-a-no-code-rag-pipeline-configuration-one-click-rag-code-generation-tool-powered-b6e8eeb70089', 'document_title': 'Based on the provided context, I would suggest the following comprehensive title:\n\n"RAGArch: A No-Code RAG Pipeline Configuration and One-Click Code Generation Tool for Streamlined AI Development, File Handling, and Large Language Model Selection for Natural Language Processing"\n\nThis title captures the main entities and themes present in the context, including:\n\n* RAGArch: the tool itself\n* No-code pipeline configuration\n* One-click code generation\n* AI development\n* File handling\n* Large Language Model selection\n* Natural Language Processing (NLP)\n\nThis title provides a clear and concise overview of the document\'s content, making it easy for readers to understand the main topics and themes discussed.'}<br>

---

**`Source Node 2/2`**

**Node ID:** b9ae64b7-7f37-4523-ae89-7b2c9c6da607<br>**Similarity:** -5.767147541046143<br>**Text:** def   generate_rag_pipeline ( file, llm, embed_model, node_parser, response_mode, vector_store ):...<br>**Metadata:** {'title': 'RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Powered by LlamaIndex', 'author': 'Harshad Suryawanshi', 'date': 'Feb 2, 2024', 'tags': 'Rag, No Code, Llamaindex, OpenAI, Code Generation', 'url': 'https://www.llamaindex.ai/blog/ragarch-building-a-no-code-rag-pipeline-configuration-one-click-rag-code-generation-tool-powered-b6e8eeb70089', 'document_title': 'Based on the provided context, I would suggest the following comprehensive title:\n\n"RAGArch: A No-Code RAG Pipeline Configuration and One-Click Code Generation Tool for Streamlined AI Development, File Handling, and Large Language Model Selection for Natural Language Processing"\n\nThis title captures the main entities and themes present in the context, including:\n\n* RAGArch: the tool itself\n* No-code pipeline configuration\n* One-click code generation\n* AI development\n* File handling\n* Large Language Model selection\n* Natural Language Processing (NLP)\n\nThis title provides a clear and concise overview of the document\'s content, making it easy for readers to understand the main topics and themes discussed.'}<br>

{'cb7da65e-0eae-42f1-81a6-1a50c6e0e9a7': {'title': 'RAGArch: Building a No-Code RAG Pipeline Configuration & One-Click RAG Code Generation Tool Powered by LlamaIndex',
  'author': 'Harshad Suryawanshi',
  'date': 'Feb 2, 2024',
  'tags': 'Rag, No Code, Llamaindex, OpenAI, Code Generation',
  'url': 'https://www.llamaindex.ai/blog/ragarch-building-a-no-code-rag-pipeline-configuration-one-click-rag-code-generation-tool-powered-b6e8eeb70089',
  'document_title': 'Based on the provided context, I would suggest the following comprehensive title:\n\n"RAGArch: A No-Code RAG Pipeline Configuration and One-Click Code Generation Tool for Streamlined AI Development, File Handling, and Large Language Model Selection for Natural Language Processing"\n\nThis title captures the main entities and themes present in the context, including:\n\n* RAGArch: the tool itself\n* No-code pipeline configuration\n* One-click code generation\n* AI development\n* File handling\n* Large Language Model selection\n* N

# Evaluation

## Retrieval Evaluation

### Building synthetic evaluation dataset

In [36]:
with open(NODES_PERSIST_FP, 'rb') as f:
    nodes = pickle.load(f)

In [37]:
from llama_index.core.evaluation import generate_question_context_pairs, EmbeddingQAFinetuneDataset

In [38]:
RECREATE_RETRIEVAL_EVAL_DATASET = True
# Currently can not reuse retrieval_eval_dataset because the retrieval evaluation is based on ids
# RETRIEVAL_EVAL_DATASET_FP = f"data/001/exp_001_v3/llamaindex_blog_retrieval_eval_dataset.json"
RETRIEVAL_EVAL_DATASET_FP = f"{NOTEBOOK_CACHE_DP}/llamaindex_blog_retrieval_eval_dataset.json"
RETRIEVAL_NUM_SAMPLE_NODES = 10
RETRIEVAL_NUM_SAMPLE_NODES = min(len(nodes), RETRIEVAL_NUM_SAMPLE_NODES)
RETRIEVAL_EVAL_LLM_MODEL = 'gpt-3.5-turbo'
RETRIEVAL_EVAL_LLM_MODEL_CONFIG = {
    "temperature": 0.3
}
RETRIEVAL_NUM_QUESTIONS_PER_CHUNK = 2
if LOG_TO_MLFLOW:
    mlflow.log_param("RETRIEVAL_NUM_QUESTIONS_PER_CHUNK", RETRIEVAL_NUM_QUESTIONS_PER_CHUNK)
    mlflow.log_param("RETRIEVAL_NUM_SAMPLE_NODES", RETRIEVAL_NUM_SAMPLE_NODES)

In [39]:
if RECREATE_RETRIEVAL_EVAL_DATASET or not os.path.exists(RETRIEVAL_EVAL_DATASET_FP):
    if RETRIEVAL_NUM_SAMPLE_NODES:
        logger.info(f"Sampling {RETRIEVAL_NUM_SAMPLE_NODES} nodes for retrieval evaluation...")
        np.random.seed(41)
        retrieval_eval_nodes = np.random.choice(nodes, RETRIEVAL_NUM_SAMPLE_NODES)
    else:
        logger.info(f"Using all nodes for retrieval evaluation")
        retrieval_eval_nodes = nodes
else:
    logger.info(f"Loading retrieval_eval_nodes from {RETRIEVAL_EVAL_DATASET_FP}...")
    with open(RETRIEVAL_EVAL_DATASET_FP, 'r') as f:
        retrieval_eval_nodes = json.load(f)

2024-07-24 22:54:51.873 | INFO     | __main__:<module>:3 - Sampling 10 nodes for retrieval evaluation...


In [40]:
if RECREATE_RETRIEVAL_EVAL_DATASET or not os.path.exists(RETRIEVAL_EVAL_DATASET_FP):
    # Use good model to generate the eval dataset
    from llama_index.llms.openai import OpenAI
    retrieval_eval_llm = OpenAI(model=RETRIEVAL_EVAL_LLM_MODEL, **RETRIEVAL_EVAL_LLM_MODEL_CONFIG)

    logger.info(f"Creating new synthetic retrieval eval dataset...")
    retrieval_eval_dataset = generate_question_context_pairs(
        retrieval_eval_nodes, llm=retrieval_eval_llm, num_questions_per_chunk=RETRIEVAL_NUM_QUESTIONS_PER_CHUNK
    )
    retrieval_eval_dataset.save_json(RETRIEVAL_EVAL_DATASET_FP)
else:
    logger.info(f"Loading existing synthetic retrieval eval dataset at {RETRIEVAL_EVAL_DATASET_FP}...")
    retrieval_eval_dataset = EmbeddingQAFinetuneDataset.from_json(RETRIEVAL_EVAL_DATASET_FP)

2024-07-24 22:54:53.806 | INFO     | __main__:<module>:6 - Creating new synthetic retrieval eval dataset...
 40%|████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                          | 4/10 [00:06<00:08,  1.48s/it]/home/dvquys/frostmourne/study/vietai-genai03/assignment1/.venv/lib/python3.11/site-packages/llama_index/core/llama_dataset/legacy/embedding.py:99: UserWarning: Fewer questions generated (1) than requested (2).
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.55s/it]


In [41]:
retrieval_eval_dataset.queries

{'91843b68-ac1a-4878-8df4-9a82f0fe9408': 'Explain the process of setting up custom query engines for a ComposableGraph using the provided code snippets. How does the TransformQueryEngine and DecomposeQueryTransform contribute to this setup?',
 '469bd344-561b-4c25-9f49-f9a5da7dd0eb': 'Compare and contrast the functionality of the GPT-3 ReAct agent, GPT-4 ReAct agent, and Simple Router agent in the context of querying a ComposableGraph. How do the tools used by each agent differ in their approach to accessing information from the graph?',
 'f7f7c25a-d3f8-49fe-98de-a19a5a430741': 'How does the process of Named Entity Recognition (NER) contribute to the personalization of news recommendations in NewsGPT? Explain the role of the dslim/bert-base-NER model from HuggingFace in this process.',
 '99b6df62-f04b-4475-a166-a5ce919485a7': 'Describe the recommendation pipeline for personalization in NewsGPT after users sign up and log in. Include details on how user preferences and activities are uti

### Evaluate

In [42]:
from llama_index.core.evaluation import RetrieverEvaluator

In [43]:
RETRIEVAL_METRICS = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    RETRIEVAL_METRICS, retriever=retriever
)

retrieval_eval_results = await retriever_evaluator.aevaluate_dataset(retrieval_eval_dataset)

> Top 5 nodes:
> [Node 378f3d9c-b23a-48cd-8a98-9cd5f69b9ddb] [Similarity score:             0.69874] Using the previous example, output = p.run(topic="YC")
# output type is Response
type(output) If ...
> [Node 0ff41e8b-9bfc-47b7-abd3-0096fd05d382] [Similarity score:             0.694699] If any entities are found, we iterate and execute the vector context retriever for each entity. O...
> [Node f8126886-be98-4eba-91a6-e198e64e7fd2] [Similarity score:             0.689692] graph_store.structured_query( """
CREATE VECTOR INDEX entity IF NOT EXISTS
FOR (m:`__Entity__`)
O...
> [Node 3cbfcd7d-bdc8-45f4-80f7-3db9245da480] [Similarity score:             0.689627] def   init ( 
        self,
         ## vector context retriever params 
        embed_model:  Op...
> [Node d295f976-4218-46ca-8b10-4eb9d415f401] [Similarity score:             0.688716] Customizing property graph index in LlamaIndex
Learn how to implement entity deduplication and cu...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 378f3d9c-b23a-48cd-8a98-9cd5f69b9ddb] [Similarity score:             0.69874] Using the previous example, output = p.run(topic="YC")
# output type is Response
type(output) If ...
> [Node 0ff41e8b-9bfc-47b7-abd3-0096fd05d382] [Similarity score:             0.694699] If any entities are found, we iterate and execute the vector context retriever for each entity. O...
> [Node f8126886-be98-4eba-91a6-e198e64e7fd2] [Similarity score:             0.689692] graph_store.structured_query( """
CREATE VECTOR INDEX entity IF NOT EXISTS
FOR (m:`__Entity__`)
O...
> [Node 3cbfcd7d-bdc8-45f4-80f7-3db9245da480] [Similarity score:             0.689627] def   init ( 
        self,
         ## vector context retriever params 
        embed_model:  Op...
> [Node d295f976-4218-46ca-8b10-4eb9d415f401] [Similarity score:             0.688716] Customizing property graph index in LlamaIndex
Learn how to implement entity deduplication and cu...


> Top 5 nodes:
> [Node 8762af05-573e-4218-8c37-61c103ff5c50] [Similarity score:             0.741513] agent_chain.run(input="How much cash did Uber have in sept 2022?") We see that the agent makes tw...
> [Node b121f089-28e9-4bb6-bf18-35bde674ac74] [Similarity score:             0.734254] High-Level Findings The high-level finding is that  less sophisticated agents need more constrain...
> [Node d7b1f3dc-479b-45f5-af9b-e6c4afcffede] [Similarity score:             0.728069] )
query_tool_graph = QueryEngineTool.from_defaults(
    query_engine=g_engine,
    description=f"...
> [Node 0118fb96-b70f-4363-83ed-f5161c3672c2] [Similarity score:             0.715617] # define indices
march_index = GPTVectorStoreIndex.from_documents(march_2022)
june_index = GPTVec...
> [Node d42ced9c-0dc7-44e6-b164-34bb9547da34] [Similarity score:             0.715541] Dumber LLM Agents Need More Constraints and Better Tools
Summary In this article, we compare how ...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 8762af05-573e-4218-8c37-61c103ff5c50] [Similarity score:             0.741513] agent_chain.run(input="How much cash did Uber have in sept 2022?") We see that the agent makes tw...
> [Node b121f089-28e9-4bb6-bf18-35bde674ac74] [Similarity score:             0.734254] High-Level Findings The high-level finding is that  less sophisticated agents need more constrain...
> [Node d7b1f3dc-479b-45f5-af9b-e6c4afcffede] [Similarity score:             0.728069] )
query_tool_graph = QueryEngineTool.from_defaults(
    query_engine=g_engine,
    description=f"...
> [Node 0118fb96-b70f-4363-83ed-f5161c3672c2] [Similarity score:             0.715617] # define indices
march_index = GPTVectorStoreIndex.from_documents(march_2022)
june_index = GPTVec...
> [Node d42ced9c-0dc7-44e6-b164-34bb9547da34] [Similarity score:             0.715541] Dumber LLM Agents Need More Constraints and Better Tools
Summary In this article, we compare how ...


> Top 5 nodes:
> [Node fe557b79-4c6f-4b13-9f69-38c82723ca06] [Similarity score:             0.79376] After the preprocessing of the data, we collect the metadata, including keywords, NER results, su...
> [Node e46a803e-c18b-4dea-99e0-182a48c60339] [Similarity score:             0.763455] Sophisticated  Named-Entity Recognition, Text Embedding with OpenAI API,  and  asynchronous artic...
> [Node 6a47aa8c-c22a-4cf0-9069-42b50a317d7e] [Similarity score:             0.743386] Chat with Article Powered by LlamaIndex LlamaIndex Pipeline Unlike other news aggregator apps, Ne...
> [Node ba03f8e0-20c5-49e9-a934-edad75b471c0] [Similarity score:             0.740407] NewsGPT(Neotice): Summarize news articles with LlamaIndex — Hackathon winning app
We’re excited t...
> [Node 65b19540-5b23-480a-8883-e0c59736b599] [Similarity score:             0.739633] from  llama_index  import  ServiceContext

st.session_state[ "service_context" ] = ServiceContext...


...

DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node b220413a-0ac9-43fd-9ffa-1eede85f5efa] [Similarity score:             0.710011] from  llama_index.vector_stores  import  ZepVectorStore
zep_api_url =  "http://localhost:8000" 
z...
> [Node 06a2f9bc-9a47-4363-9010-38a896efe735] [Similarity score:             0.685134] print ( str (response)) But one of the most signal examples of this kind, of which we are aware, ...
> [Node b3f21c8e-d13a-4b5f-9272-6d715ca85ed7] [Similarity score:             0.684357] Simply create a Timescale Vector vector store and add the  data nodes  you want to query as shown...
> [Node 78161534-e09e-4ae1-a0ff-a9b703ae54f0] [Similarity score:             0.678696] from pypdf import PdfReader from llama_index.schema import Document def process_pdf(pdf):     fil...
> [Node 4479325e-d44c-40f4-8982-cc3309945418] [Similarity score:             0.673028] See our usage example below as well. Usage Example To build a  VectorStoreIndex  and then query i...


> Top 5 nodes:
> [Node 550a4330-b77b-4a95-976f-8c7ac0e984c1] [Similarity score:             0.839029] LLM-based retrieval is orders of magnitude slower than embedding-based retrieval. Embedding searc...
> [Node 36e508bc-cb26-43b5-9ab4-ac344a03bfcb] [Similarity score:             0.831561] Yet for a variety of reasons, embedding-based retrieval can be imprecise and return irrelevant co...
> [Node 124ae013-39ea-49ab-a870-f5f98d46a71d] [Similarity score:             0.808496] We’ve added some initial functionality to help support LLM-augmented retrieval pipelines, but of ...
> [Node 511ef310-c1e9-4b5f-b760-3a3e97512825] [Similarity score:             0.807362] Using LLM’s for Retrieval and Reranking
Summary This blog post outlines some of the core abstract...
> [Node 17fbe4dd-c34d-4c03-9f71-bd68136e2caf] [Similarity score:             0.803881] We see that in embedding-based retrieval, the top two texts contain semantics of the car crash bu...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 550a4330-b77b-4a95-976f-8c7ac0e984c1] [Similarity score:             0.839029] LLM-based retrieval is orders of magnitude slower than embedding-based retrieval. Embedding searc...
> [Node 36e508bc-cb26-43b5-9ab4-ac344a03bfcb] [Similarity score:             0.831561] Yet for a variety of reasons, embedding-based retrieval can be imprecise and return irrelevant co...
> [Node 124ae013-39ea-49ab-a870-f5f98d46a71d] [Similarity score:             0.808496] We’ve added some initial functionality to help support LLM-augmented retrieval pipelines, but of ...
> [Node 511ef310-c1e9-4b5f-b760-3a3e97512825] [Similarity score:             0.807362] Using LLM’s for Retrieval and Reranking
Summary This blog post outlines some of the core abstract...
> [Node 17fbe4dd-c34d-4c03-9f71-bd68136e2caf] [Similarity score:             0.803881] We see that in embedding-based retrieval, the top two texts contain semantics of the car crash bu...


> Top 5 nodes:
> [Node 550a4330-b77b-4a95-976f-8c7ac0e984c1] [Similarity score:             0.832421] LLM-based retrieval is orders of magnitude slower than embedding-based retrieval. Embedding searc...
> [Node 36e508bc-cb26-43b5-9ab4-ac344a03bfcb] [Similarity score:             0.829715] Yet for a variety of reasons, embedding-based retrieval can be imprecise and return irrelevant co...
> [Node 124ae013-39ea-49ab-a870-f5f98d46a71d] [Similarity score:             0.822108] We’ve added some initial functionality to help support LLM-augmented retrieval pipelines, but of ...
> [Node 17fbe4dd-c34d-4c03-9f71-bd68136e2caf] [Similarity score:             0.814088] We see that in embedding-based retrieval, the top two texts contain semantics of the car crash bu...
> [Node 511ef310-c1e9-4b5f-b760-3a3e97512825] [Similarity score:             0.811899] Using LLM’s for Retrieval and Reranking
Summary This blog post outlines some of the core abstract...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 550a4330-b77b-4a95-976f-8c7ac0e984c1] [Similarity score:             0.832421] LLM-based retrieval is orders of magnitude slower than embedding-based retrieval. Embedding searc...
> [Node 36e508bc-cb26-43b5-9ab4-ac344a03bfcb] [Similarity score:             0.829715] Yet for a variety of reasons, embedding-based retrieval can be imprecise and return irrelevant co...
> [Node 124ae013-39ea-49ab-a870-f5f98d46a71d] [Similarity score:             0.822108] We’ve added some initial functionality to help support LLM-augmented retrieval pipelines, but of ...
> [Node 17fbe4dd-c34d-4c03-9f71-bd68136e2caf] [Similarity score:             0.814088] We see that in embedding-based retrieval, the top two texts contain semantics of the car crash bu...
> [Node 511ef310-c1e9-4b5f-b760-3a3e97512825] [Similarity score:             0.811899] Using LLM’s for Retrieval and Reranking
Summary This blog post outlines some of the core abstract...


In [44]:
def display_results(name, eval_results, metrics=['hit_rate', 'mrr'], include_cohere_rerank=False):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    if include_cohere_rerank:
        crr_relevancy = full_df["cohere_rerank_relevancy"].mean()
        columns.update({"cohere_rerank_relevancy": [crr_relevancy]})

    metric_df = pd.DataFrame(columns)

    return metric_df

In [45]:
metric_prefix = f"top_{RETRIEVAL_TOP_K}_retrieval_eval"
retrieval_eval_results_df = display_results(metric_prefix, retrieval_eval_results, metrics=RETRIEVAL_METRICS)
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_5_retrieval_eval,0.631579,0.55,0.126316,0.631579,0.55,0.193099


In [46]:
if LOG_TO_MLFLOW:
    for metric, metric_value in retrieval_eval_results_df.to_dict(orient='records')[0].items():
        if metric in RETRIEVAL_METRICS:
            mlflow.log_metric(f"{metric_prefix}_{metric}", metric_value)

### Manually curated dataset
Ref: https://docs.llamaindex.ai/en/stable/module_guides/evaluating/usage_pattern_retrieval/

In [47]:
MANUAL_EVAL_QA = [
("What are key features of llama-agents?",
"""
Key features of llama-agents are:
1. Distributed Service Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks.
2. Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue.
3. Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an “agentic orchestrator” that decides which agents are relevant to the task.
4. Ease of deployment: launch, scale and monitor each agent and your control plane independently.
5. Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent service
"""
),
("What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?",
"""
Retrieval System and Response Generation.
"""
),
("What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system?",
"""
Hit rate and Mean Reciprocal Rank (MRR)

Hit Rate: Hit rate calculates the fraction of queries where the correct answer is found within the top-k retrieved documents. In simpler terms, it’s about how often our system gets it right within the top few guesses.

Mean Reciprocal Rank (MRR): For each query, MRR evaluates the system’s accuracy by looking at the rank of the highest-placed relevant document. Specifically, it’s the average of the reciprocals of these ranks across all the queries. So, if the first relevant document is the top result, the reciprocal rank is 1; if it’s second, the reciprocal rank is 1/2, and so on.
"""
)
]

## Response Evaluation
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/downloading_llama_datasets/

In [48]:
def evaluate_labelled_rag_dataset(response_eval_dataset, response_eval_prediction_dataset, dataset_name="synthetic", batch_size=8, judge_model='gpt-3.5-turbo', cache_dp='.'):
    # Instantiate the judges
    judges = {
        "correctness": CorrectnessEvaluator(
            llm=OpenAI(temperature=0, model=judge_model),
        ),
        "relevancy": RelevancyEvaluator(
            llm=OpenAI(temperature=0, model=judge_model),
        ),
        "faithfulness": FaithfulnessEvaluator(
            llm=OpenAI(temperature=0, model=judge_model),
        ),
        "semantic_similarity": SemanticSimilarityEvaluator(),
    }

    # Initialize evaluations dictionary
    evals = {
        "correctness": [],
        "relevancy": [],
        "faithfulness": [],
        "contexts": [],
    }

    # Evaluate each prediction
    for example, prediction in tqdm(
        zip(response_eval_dataset.examples, response_eval_prediction_dataset.predictions)
    ):
        correctness_result = judges["correctness"].evaluate(
            query=example.query,
            response=prediction.response,
            reference=example.reference_answer,
        )

        relevancy_result = judges["relevancy"].evaluate(
            query=example.query,
            response=prediction.response,
            contexts=prediction.contexts,
        )

        faithfulness_result = judges["faithfulness"].evaluate(
            query=example.query,
            response=prediction.response,
            contexts=prediction.contexts,
        )

        evals["correctness"].append(correctness_result)
        evals["relevancy"].append(relevancy_result)
        evals["faithfulness"].append(faithfulness_result)
        evals["contexts"].append(prediction.contexts)

    # Save evaluations to JSON
    evaluations_objects = {
        "correctness": [e.dict() for e in evals["correctness"]],
        "faithfulness": [e.dict() for e in evals["faithfulness"]],
        "relevancy": [e.dict() for e in evals["relevancy"]],
        "contexts": evals['contexts'],
    }

    with open(f"{cache_dp}/{dataset_name}_evaluations.json", "w") as json_file:
        json.dump(evaluations_objects, json_file)

    # Generate evaluation results DataFrames
    deep_eval_correctness_df, mean_correctness_df = get_eval_results_df(
        ["base_rag"] * len(evals["correctness"]),
        evals["correctness"],
        metric="correctness",
    )
    deep_eval_relevancy_df, mean_relevancy_df = get_eval_results_df(
        ["base_rag"] * len(evals["relevancy"]),
        evals["relevancy"],
        metric="relevancy",
    )
    deep_eval_faithfulness_df, mean_faithfulness_df = get_eval_results_df(
        ["base_rag"] * len(evals["faithfulness"]),
        evals["faithfulness"],
        metric="faithfulness",
    )

    mean_scores_df = pd.concat(
        [
            mean_correctness_df.reset_index(),
            mean_relevancy_df.reset_index(),
            mean_faithfulness_df.reset_index(),
        ],
        axis=0,
        ignore_index=True,
    )
    mean_scores_df = mean_scores_df.set_index("index")
    mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])

    deep_eval_df = pd.concat([
        deep_eval_correctness_df[['query', 'answer']],
        deep_eval_relevancy_df[['scores']].rename(columns={'scores': 'relevancy_score'}),
        deep_eval_correctness_df[['scores']].rename(columns={'scores': 'correctness_score'}),
        deep_eval_faithfulness_df[['scores']].rename(columns={'scores': 'faithfulness_score'}),
        pd.Series(evals['contexts'], name='contexts')
    ], axis=1)

    return mean_scores_df, deep_eval_df

### Generate synthetic Llama Dataset

In [49]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core.llama_dataset import LabeledRagDataset
from llama_index.core.evaluation import (
    CorrectnessEvaluator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    SemanticSimilarityEvaluator,
)
from llama_index.core.evaluation.notebook_utils import get_eval_results_df

In [50]:
RECREATE_SYNTHETIC_EVAL_DATASET = False
# RESPONSE_EVAL_DATASET_FP = f"{NOTEBOOK_CACHE_DP}/llamaindex_blog_response_eval_dataset.json"
RESPONSE_EVAL_DATASET_FP = f"data/001/exp_001_v3/llamaindex_blog_response_eval_dataset.json"
RESPONSE_EVAL_LLM_MODEL = 'gpt-3.5-turbo'
RESPONSE_EVAL_LLM_MODEL_CONFIG = {
    "temperature": 0.3
}
SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK = 2
RESPONSE_NUM_SAMPLE_DOCUMENTS = 10
RESPONSE_NUM_SAMPLE_DOCUMENTS = min(len(documents), RESPONSE_NUM_SAMPLE_DOCUMENTS)

if LOG_TO_MLFLOW:
    mlflow.log_param("SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK", SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK)
    mlflow.log_param("RESPONSE_EVAL_LLM_MODEL", RESPONSE_EVAL_LLM_MODEL)
    mlflow.log_param("RESPONSE_NUM_SAMPLE_DOCUMENTS", RESPONSE_NUM_SAMPLE_DOCUMENTS)
    for k, v in RESPONSE_EVAL_LLM_MODEL_CONFIG.items():
        mlflow.log_param(f"RESPONSE_EVAL_LLM_MODEL_CONFIG__{k}", v)

In [51]:
if RESPONSE_NUM_SAMPLE_DOCUMENTS:
    logger.info(f"Sampling {RESPONSE_NUM_SAMPLE_DOCUMENTS} documents for response evaluation...")
    np.random.seed(41)
    response_eval_documents = np.random.choice(documents, RESPONSE_NUM_SAMPLE_DOCUMENTS)
else:
    logger.info(f"Using all documents for retrieval evaluation")
    response_eval_documents = documents

2024-07-24 22:56:51.188 | INFO     | __main__:<module>:2 - Sampling 10 documents for response evaluation...


In [52]:
if RECREATE_SYNTHETIC_EVAL_DATASET or not os.path.exists(RESPONSE_EVAL_DATASET_FP):
    logger.info(f"Creating synthetic response eval dataset...")
    # Use good model to generate the eval dataset
    response_eval_llm = OpenAI(model=RESPONSE_EVAL_LLM_MODEL, **RESPONSE_EVAL_LLM_MODEL_CONFIG)

    # instantiate a DatasetGenerator
    response_dataset_generator = RagDatasetGenerator.from_documents(
        response_eval_documents,
        llm=response_eval_llm,
        num_questions_per_chunk=SYNTHETIC_RESPONSE_NUM_QUESTIONS_PER_CHUNK,  # set the number of questions per nodes
        show_progress=True,
        workers=(os.cpu_count() - 1)
    )

    synthetic_response_eval_dataset = response_dataset_generator.generate_dataset_from_nodes()

    synthetic_response_eval_dataset.save_json(RESPONSE_EVAL_DATASET_FP)
else:
    logger.info(f"Loading existing synthetic response eval dataset at {RESPONSE_EVAL_DATASET_FP}...")
    synthetic_response_eval_dataset = LabeledRagDataset.from_json(RESPONSE_EVAL_DATASET_FP)

2024-07-24 22:56:53.086 | INFO     | __main__:<module>:19 - Loading existing synthetic response eval dataset at data/001/exp_001_v3/llamaindex_blog_response_eval_dataset.json...


In [54]:
synthetic_response_eval_prediction_dataset = await synthetic_response_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=8, show_progress=True
)


Batch processing of predictions:   0%|                                                                                                                                                                                                               | 0/8 [00:00<?, ?it/s]

> Top 5 nodes:
> [Node c4e47e3e-d056-4ef1-bff1-9be8b3180892] [Similarity score:             0.854614] Docs ,  Tweet . RA-DIT:  We drew inspiration from the RA-DIT paper, which introduced LLM fine-tun...
> [Node 3ad42e4f-17b2-4d0d-8ba2-297acbc1adf0] [Similarity score:             0.853496] This approach overcomes the limitations of fine-tuning LLMs and provides a more cost-effective, u...
> [Node 1b7d4deb-9abd-46f3-8e9f-9d12f81b9918] [Similarity score:             0.846551] Building Scalable RAG Applications with LlamaIndex and Zilliz Cloud Pipelines
Introduction We are...
> [Node 6212e071-f6ee-454f-bb70-e8c7e1704811] [Similarity score:             0.841555] Simplify your RAG application architecture with LlamaIndex + PostgresML
We’re happy to announce t...
> [Node 1ce6a371-f31a-456c-a184-7b970012fce8] [Similarity score:             0.841511] LlamaIndex update 2023–10–10
Here’s our weekly look at developments across the LLM space and RAG ...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node c4e47e3e-d056-4ef1-bff1-9be8b3180892] [Similarity score:             0.854614] Docs ,  Tweet . RA-DIT:  We drew inspiration from the RA-DIT paper, which introduced LLM fine-tun...
> [Node 3ad42e4f-17b2-4d0d-8ba2-297acbc1adf0] [Similarity score:             0.853496] This approach overcomes the limitations of fine-tuning LLMs and provides a more cost-effective, u...
> [Node 1b7d4deb-9abd-46f3-8e9f-9d12f81b9918] [Similarity score:             0.846551] Building Scalable RAG Applications with LlamaIndex and Zilliz Cloud Pipelines
Introduction We are...
> [Node 6212e071-f6ee-454f-bb70-e8c7e1704811] [Similarity score:             0.841555] Simplify your RAG application architecture with LlamaIndex + PostgresML
We’re happy to announce t...
> [Node 1ce6a371-f31a-456c-a184-7b970012fce8] [Similarity score:             0.841511] LlamaIndex update 2023–10–10
Here’s our weekly look at developments across the LLM space and RAG ...


> Top 5 nodes:
> [Node f85e8372-aade-4846-aa4c-d8c874a1dab6] [Similarity score:             0.807304] LlamaIndex Newsletter 2024-03-19
Greetings, LlamaIndex enthusiasts! 🦙 Welcome to another exciting...
> [Node 92348901-97fe-4a64-918c-7f706507f4a6] [Similarity score:             0.80093] In V0.9, we have  flattened  the entire interface into a single  TransformComponent  abstraction,...
> [Node d10877d0-5a7a-43a2-9019-12235b516604] [Similarity score:             0.800342] Launching the first GenAI-native document parsing platform
Our mission at LlamaIndex is to connec...
> [Node 20d10f16-820e-46a4-9c64-233181488e36] [Similarity score:             0.799856] LlamaParse Demo. Given a PDF file, returns a parsed markdown file that maintains semantic structu...
> [Node e41335df-9543-4738-8397-dbb472a1db68] [Similarity score:             0.796163] from  llama_parse  import  LlamaParse

parser = LlamaParse(
    api_key= "llx-..." ,   # can also...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node f85e8372-aade-4846-aa4c-d8c874a1dab6] [Similarity score:             0.807304] LlamaIndex Newsletter 2024-03-19
Greetings, LlamaIndex enthusiasts! 🦙 Welcome to another exciting...
> [Node 92348901-97fe-4a64-918c-7f706507f4a6] [Similarity score:             0.80093] In V0.9, we have  flattened  the entire interface into a single  TransformComponent  abstraction,...
> [Node d10877d0-5a7a-43a2-9019-12235b516604] [Similarity score:             0.800342] Launching the first GenAI-native document parsing platform
Our mission at LlamaIndex is to connec...
> [Node 20d10f16-820e-46a4-9c64-233181488e36] [Similarity score:             0.799856] LlamaParse Demo. Given a PDF file, returns a parsed markdown file that maintains semantic structu...
> [Node e41335df-9543-4738-8397-dbb472a1db68] [Similarity score:             0.796163] from  llama_parse  import  LlamaParse

parser = LlamaParse(
    api_key= "llx-..." ,   # can also...


...

DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 54e11644-4e1e-46e1-8a09-088f9ef29c81] [Similarity score:             0.812297] LlamaIndex newsletter 2023–10–24
Hello Llama Fans 🦙! Welcome back to our newsletter covering new ...
> [Node ebeead3d-3bf3-4d34-8d5a-3414ce1ec656] [Similarity score:             0.812287] Docs . ✨ Feature Releases and Enhancements: We launched a partnership with Google Gemini, offerin...
> [Node 09cb975c-e82f-492e-82f1-8200ab9eb8a8] [Similarity score:             0.809868] Tweet ,  Docs . SQLRetriever:  We introduce SQLRetriever, merging Text-to-SQL and RAG, enabling a...
> [Node 13277cc5-5ff9-4c03-808c-18275f7851f3] [Similarity score:             0.806972] Register for free! ✨ Feature Releases and Enhancements: We introduced the LlamaIndex 0.9 version ...
> [Node ea376d9c-46a4-41da-b6fc-78a80e20b2ac] [Similarity score:             0.806964] This expands beyond its existing domain of loaders, tools, packs, and datasets, to include LLMs, ...


> Top 5 nodes:
> [Node 5165720d-0448-4bf9-93f5-67b87c81ad34] [Similarity score:             0.818749] We’ll walk through a simple  Llama Pack  that gives the user a RAG pipeline setup with Voyage AI ...
> [Node b63739f3-c42a-46e5-870c-b7c42412ea4e] [Similarity score:             0.783143] They can be downloaded either through our  llama_index  Python library or the CLI in  one line of...
> [Node 87d250f6-94cc-4995-8b66-10e8f341abd0] [Similarity score:             0.780944] Let’s take a look at the downloaded pack in  voyage_pack/base.py  , and swap out the OpenAI LLM f...
> [Node 7829c363-51b9-4c2e-879d-e88ac40511be] [Similarity score:             0.744008] Introducing Llama Packs
Today we’re excited to introduce  Llama Packs 🦙📦—  a community-driven hub...
> [Node f1b2a2b8-4296-42b6-9b65-5790705632d0] [Similarity score:             0.74286] Take a look at this folder for a full set of examples:  https://github.com/run-llama/llama-hub/tr...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 5165720d-0448-4bf9-93f5-67b87c81ad34] [Similarity score:             0.818749] We’ll walk through a simple  Llama Pack  that gives the user a RAG pipeline setup with Voyage AI ...
> [Node b63739f3-c42a-46e5-870c-b7c42412ea4e] [Similarity score:             0.783143] They can be downloaded either through our  llama_index  Python library or the CLI in  one line of...
> [Node 87d250f6-94cc-4995-8b66-10e8f341abd0] [Similarity score:             0.780944] Let’s take a look at the downloaded pack in  voyage_pack/base.py  , and swap out the OpenAI LLM f...
> [Node 7829c363-51b9-4c2e-879d-e88ac40511be] [Similarity score:             0.744008] Introducing Llama Packs
Today we’re excited to introduce  Llama Packs 🦙📦—  a community-driven hub...
> [Node f1b2a2b8-4296-42b6-9b65-5790705632d0] [Similarity score:             0.74286] Take a look at this folder for a full set of examples:  https://github.com/run-llama/llama-hub/tr...


> Top 5 nodes:
> [Node 2cdcfcec-59bb-42f5-a719-84ba0717096b] [Similarity score:             0.814157] Now, you can easily process complex documents like PDFs, PPTX, and Markdown files, ensuring clean...
> [Node 09cb975c-e82f-492e-82f1-8200ab9eb8a8] [Similarity score:             0.806569] Tweet ,  Docs . SQLRetriever:  We introduce SQLRetriever, merging Text-to-SQL and RAG, enabling a...
> [Node a8e236c0-25db-4dab-b602-eeedc16426f7] [Similarity score:             0.80018] Guide  to Building an Agent in LlamaIndex: Our comprehensive guide which covers building a basic ...
> [Node 0cbcc61e-85c6-4411-95a3-71a58d28aa34] [Similarity score:             0.799392] ✍️ Tutorials: Akash Mathur ’s  tutorial  on Data Management in LlamaIndex: Featuring LlamaCloud a...
> [Node 300ca4c1-82bf-4385-82a3-3b7003c12c29] [Similarity score:             0.798582] Cookbook ,  Tweet . 💡 Demos: Automating Code Reviews,  project  by  Composio  with LlamaIndex aut...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 2cdcfcec-59bb-42f5-a719-84ba0717096b] [Similarity score:             0.814157] Now, you can easily process complex documents like PDFs, PPTX, and Markdown files, ensuring clean...
> [Node 09cb975c-e82f-492e-82f1-8200ab9eb8a8] [Similarity score:             0.806569] Tweet ,  Docs . SQLRetriever:  We introduce SQLRetriever, merging Text-to-SQL and RAG, enabling a...
> [Node a8e236c0-25db-4dab-b602-eeedc16426f7] [Similarity score:             0.80018] Guide  to Building an Agent in LlamaIndex: Our comprehensive guide which covers building a basic ...
> [Node 0cbcc61e-85c6-4411-95a3-71a58d28aa34] [Similarity score:             0.799392] ✍️ Tutorials: Akash Mathur ’s  tutorial  on Data Management in LlamaIndex: Featuring LlamaCloud a...
> [Node 300ca4c1-82bf-4385-82a3-3b7003c12c29] [Similarity score:             0.798582] Cookbook ,  Tweet . 💡 Demos: Automating Code Reviews,  project  by  Composio  with LlamaIndex aut...

Batch 

In [55]:
synthetic_mean_scores_df, synthetic_deep_eval_df = evaluate_labelled_rag_dataset(
    synthetic_response_eval_dataset,
    synthetic_response_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=RESPONSE_EVAL_LLM_MODEL,
    cache_dp=NOTEBOOK_CACHE_DP
)

0it [00:00, ?it/s]

> Adding chunk: Querying a network of knowledge with llama-inde...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Querying a network of knowledge with llama-inde...


> Adding chunk: To illustrate how the llama-index-networks pack...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: To illustrate how the llama-index-networks pack...


...

DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Lakshmi Narayana  tutorial  on Agentic RAG with...


> Adding chunk: Ravi Theja   tutorial  on showcasing RAG with L...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Ravi Theja   tutorial  on showcasing RAG with L...


> Adding chunk: Lakshmi Narayana  tutorial  on Agentic RAG with...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Lakshmi Narayana  tutorial  on Agentic RAG with...


In [56]:
synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.020833
mean_relevancy_score,0.966667
mean_faithfulness_score,0.933333


In [57]:
synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,How does the new feature released by LlamaInde...,"\nThe new feature released by LlamaIndex, llam...",1.0,4.0,1.0,[Querying a network of knowledge with llama-in...
1,Discuss the advancements made by LlamaIndex in...,\nLlamaIndex has made significant advancements...,1.0,4.0,1.0,"[LlamaIndex Newsletter 2024-03-19\nGreetings, ..."
2,Explain the three main sections of the OpenAI ...,\nThe OpenAI Cookbook for evaluating RAG syste...,1.0,NaN,1.0,[OpenAI Cookbook: Evaluating RAG systems\nWe’r...
3,How does the OpenAI Cookbook suggest evaluatin...,\nThe OpenAI Cookbook suggests evaluating the ...,1.0,NaN,1.0,[OpenAI Cookbook: Evaluating RAG systems\nWe’r...
4,How has LlamaIndex evolved over the past year ...,\nLlamaIndex's community has grown significant...,1.0,NaN,1.0,[LlamaIndex has evolved into a broad toolkit c...
5,Can you explain the significance of the Retrie...,\nThe significance of Retrieval-Augmented Gene...,1.0,4.0,1.0,[LlamaIndex update 2023–10–10\nHere’s our week...
6,How does the partnership with Google Gemini be...,\nThe partnership with Google Gemini offers da...,1.0,4.5,1.0,[Docs . ✨ Feature Releases and Enhancements: W...
7,Describe the Multi-Doc SEC 10Q Dataset launche...,"\nThe Multi-Doc SEC 10Q Dataset, launched by T...",1.0,4.5,0.0,[Long-Context Embedding Models: Explore model...
8,How does the MemoryCache project by Mozilla ut...,\nThe MemoryCache project by Mozilla is not me...,0.0,2.0,0.0,[Example question: Who is the most recent UK p...
9,Discuss the significance of integrating Na2SQL...,\nThe integration of Na2SQL with Llama Index i...,1.0,NaN,1.0,[Transforming Natural Language to SQL and Insi...


In [58]:
if LOG_TO_MLFLOW:
    for k, v in synthetic_mean_scores_df.T.to_dict(orient='records')[0].items():
        mlflow.log_metric(f"synthetic_response_eval__{k}", v)
    synthetic_deep_eval_df.to_html(f"{NOTEBOOK_CACHE_DP}/synthetic_deep_eval_df.html")
    mlflow.log_artifact(f"{NOTEBOOK_CACHE_DP}/synthetic_deep_eval_df.html", "synthetic_deep_eval_df")

### Manually curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [59]:
from llama_index.core.llama_dataset import LabelledRagDataset, LabelledRagDataExample, CreatedBy, CreatedByType

examples = []

for question, expected_anwser in MANUAL_EVAL_QA:
    example = LabelledRagDataExample(
        query=question,
        query_by=CreatedBy(type=CreatedByType.HUMAN),
        reference_answer=expected_anwser,
        reference_answer_by=CreatedBy(type=CreatedByType.HUMAN),
        reference_contexts=[],
    )
    examples.append(example)

curated_response_eval_dataset = LabelledRagDataset(examples=examples)

# save this dataset as it is required for the submission
curated_response_eval_dataset.save_json(f"{NOTEBOOK_CACHE_DP}/curated_response_eval_dataset.json")

In [60]:
curated_response_eval_prediction_dataset = await curated_response_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=8, show_progress=True
)


Batch processing of predictions:   0%|                                                                                                                                                                                                               | 0/3 [00:00<?, ?it/s]

> Top 5 nodes:
> [Node ae7cd52b-79bc-4316-bd0c-a3efbff8facd] [Similarity score:             0.828923] OpenAI Cookbook: Evaluating RAG systems
We’re excited to unveil our  OpenAI Cookbook , a guide to...
> [Node 4fe81f5b-f19e-437e-9b5b-f5acff5a45fb] [Similarity score:             0.798588] LlamaIndex Iterative Retrieval-Generator Recipe ( notebook guide ): from  llama_index.query_engin...
> [Node b9a5811a-a550-4ace-846e-449577ac58dc] [Similarity score:             0.795742] Faithfulness Evaluator  — It is useful for measuring if the response was hallucinated and measure...
> [Node 787a2ca5-23f5-422a-a9d8-227e650f7986] [Similarity score:             0.793991] total_response_time =  0 
    total_faithfulness =  0 
    total_relevancy =  0 

     # create v...
> [Node 142c9060-f370-4af1-b499-6fac0cdfd82b] [Similarity score:             0.79393] It's vital to undergo thorough testing with various sizes to find a configuration that suits the ...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node ae7cd52b-79bc-4316-bd0c-a3efbff8facd] [Similarity score:             0.828923] OpenAI Cookbook: Evaluating RAG systems
We’re excited to unveil our  OpenAI Cookbook , a guide to...
> [Node 4fe81f5b-f19e-437e-9b5b-f5acff5a45fb] [Similarity score:             0.798588] LlamaIndex Iterative Retrieval-Generator Recipe ( notebook guide ): from  llama_index.query_engin...
> [Node b9a5811a-a550-4ace-846e-449577ac58dc] [Similarity score:             0.795742] Faithfulness Evaluator  — It is useful for measuring if the response was hallucinated and measure...
> [Node 787a2ca5-23f5-422a-a9d8-227e650f7986] [Similarity score:             0.793991] total_response_time =  0 
    total_faithfulness =  0 
    total_relevancy =  0 

     # create v...
> [Node 142c9060-f370-4af1-b499-6fac0cdfd82b] [Similarity score:             0.79393] It's vital to undergo thorough testing with various sizes to find a configuration that suits the ...


> Top 5 nodes:
> [Node 19c3c775-511c-4794-bc6e-c58d061ab92b] [Similarity score:             0.795308] Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
We'...
> [Node 5a9f511a-cf5e-47e7-94ec-b30ab3f88ba6] [Similarity score:             0.782661] First we’ll bring in our dependencies and set up our control plane, which contains our LLM-powere...
> [Node 8a9ae6b8-3ebe-412c-95e7-81389137211b] [Similarity score:             0.780028] tool = FunctionTool.from_defaults(fn=get_the_secret_fact)

 # create our agents 
worker1 = Functi...
> [Node dacc1bbf-45e9-4a02-a50b-3dd1edafbaf0] [Similarity score:             0.771633] His name is Peter." )]
index = VectorStoreIndex.from_documents(docs)

 # Define a query rewrite a...
> [Node 0e96da47-6d49-4889-940c-bab9bc1305d8] [Similarity score:             0.771463] # Set up the launcher for local testing 
 from  llama_agents  import  LocalLauncher

launcher = L...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 19c3c775-511c-4794-bc6e-c58d061ab92b] [Similarity score:             0.795308] Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
We'...
> [Node 5a9f511a-cf5e-47e7-94ec-b30ab3f88ba6] [Similarity score:             0.782661] First we’ll bring in our dependencies and set up our control plane, which contains our LLM-powere...
> [Node 8a9ae6b8-3ebe-412c-95e7-81389137211b] [Similarity score:             0.780028] tool = FunctionTool.from_defaults(fn=get_the_secret_fact)

 # create our agents 
worker1 = Functi...
> [Node dacc1bbf-45e9-4a02-a50b-3dd1edafbaf0] [Similarity score:             0.771633] His name is Peter." )]
index = VectorStoreIndex.from_documents(docs)

 # Define a query rewrite a...
> [Node 0e96da47-6d49-4889-940c-bab9bc1305d8] [Similarity score:             0.771463] # Set up the launcher for local testing 
 from  llama_agents  import  LocalLauncher

launcher = L...


> Top 5 nodes:
> [Node 6f5711d0-6264-4318-8460-e140798f8fa9] [Similarity score:             0.740395] Mean Reciprocal Rank (MRR): For each query, MRR evaluates the system’s accuracy by looking at the...
> [Node 253605a1-88d3-4cba-a980-2e6acc42d01e] [Similarity score:             0.740139] Boosting RAG: Picking the Best Embedding & Reranker models
UPDATE : The pooling method for the Ji...
> [Node 39cb5226-2608-46f9-886e-42189f013b3e] [Similarity score:             0.733596] bge-large : Experiences significant improvement with rerankers, with the best results from  Coher...
> [Node bf17f2c7-8703-460b-a31a-fc002354ad6f] [Similarity score:             0.732614] Necessity of Rerankers : The data clearly indicates the significance of rerankers in refining sea...
> [Node aa6d1140-3604-4eca-ad44-3575b13e71f2] [Similarity score:             0.729997] Implemented by the user.

        """ 
         return  self._retrieve(query_bundle)

     async ...


DEBUG [llama_index.core.indices.utils] > Top 5 nodes:
> [Node 6f5711d0-6264-4318-8460-e140798f8fa9] [Similarity score:             0.740395] Mean Reciprocal Rank (MRR): For each query, MRR evaluates the system’s accuracy by looking at the...
> [Node 253605a1-88d3-4cba-a980-2e6acc42d01e] [Similarity score:             0.740139] Boosting RAG: Picking the Best Embedding & Reranker models
UPDATE : The pooling method for the Ji...
> [Node 39cb5226-2608-46f9-886e-42189f013b3e] [Similarity score:             0.733596] bge-large : Experiences significant improvement with rerankers, with the best results from  Coher...
> [Node bf17f2c7-8703-460b-a31a-fc002354ad6f] [Similarity score:             0.732614] Necessity of Rerankers : The data clearly indicates the significance of rerankers in refining sea...
> [Node aa6d1140-3604-4eca-ad44-3575b13e71f2] [Similarity score:             0.729997] Implemented by the user.

        """ 
         return  self._retrieve(query_bundle)

     async ...

Batch

In [61]:
curated_mean_scores_df, curated_deep_eval_df = evaluate_labelled_rag_dataset(
    curated_response_eval_dataset,
    curated_response_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=RESPONSE_EVAL_LLM_MODEL,
    cache_dp=NOTEBOOK_CACHE_DP
)

0it [00:00, ?it/s]

> Adding chunk: Introducing llama-agents: A Powerful Framework ...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Introducing llama-agents: A Powerful Framework ...


> Adding chunk: tool = FunctionTool.from_defaults(fn=get_the_se...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: tool = FunctionTool.from_defaults(fn=get_the_se...


...

DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Necessity of Rerankers : The data clearly indic...


> Adding chunk: bge-large : Experiences significant improvement...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: bge-large : Experiences significant improvement...


> Adding chunk: Necessity of Rerankers : The data clearly indic...


DEBUG [llama_index.core.node_parser.node_utils] > Adding chunk: Necessity of Rerankers : The data clearly indic...


In [62]:
curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,5.0
mean_relevancy_score,1.0
mean_faithfulness_score,1.0


In [63]:
with pd.option_context('display.max_colwidth', None):
    display(curated_deep_eval_df)

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are key features of llama-agents?,"\nDistributed Service-Oriented Architecture: Every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks.\n\nCommunication via Standardized API Interfaces: Interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue. Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an “agentic orchestrator” that decides which agents are relevant to the task.\n\nEase of Deployment: Launch, scale, and monitor each agent and your control plane independently.\n\n\nSources:\n- [Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems](https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems)\n\n",1.0,NaN,1.0,"[Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems\nWe're excited to announce the alpha release of llama-agents , a new open-source framework designed to simplify the process of building, iterating, and deploying multi-agent AI systems and turn your agents into production microservices. Whether you're working on complex question-answering systems, collaborative AI assistants, or distributed AI workflows, llama-agents provides the tools and structure you need to bring your ideas to life. Key Features of llama-agents Distributed Service Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks. Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue. Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an “agentic orchestrator” that decides which agents are relevant to the task. Ease of deployment: launch, scale and monitor each agent and your control plane independently. Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent service Let's dive into how you can start using llama-agents to build your own multi-agent systems. Getting Started with llama-agents First, install the framework using pip: pip install llama-agents llama-index-agent-openai Basic System Setup Here's a simple example of how to set up a basic multi-agent system using llama-agents., tool = FunctionTool.from_defaults(fn=get_the_secret_fact)\n\n # create our agents \nworker1 = FunctionCallingAgentWorker.from_tools([tool], llm=OpenAI())\nworker2 = FunctionCallingAgentWorker.from_tools([], llm=OpenAI())\nagent1 = worker1.as_agent()\nagent2 = worker2.as_agent() We turn those agents into services: agent_server_1 = AgentService(\n agent=agent1,\n message_queue=message_queue,\n description= ""Useful for getting the secret fact."" ,\n service_name= ""secret_fact_agent"" ,\n host= ""localhost"" ,\n port= 8003 \n)\nagent_server_2 = AgentService(\n agent=agent2,\n message_queue=message_queue,\n description= ""Useful for getting random dumb facts."" ,\n service_name= ""dumb_fact_agent"" ,\n host= ""localhost"" ,\n port= 8004 \n) And finally we launch each service as an independent agent. Here we’re doing them all from a single script, but each of these could be a totally separate service, launched and scaled independently: from llama_agents import ServerLauncher, CallableMessageConsumer\n\n # Additional human consumer \n def handle_result ( message ) -> None :\n print ( f""Got result:"" , message.

In [64]:
for context in curated_deep_eval_df.iloc[2]['contexts']:
    print(context)
    print('-' * 10)

bge-large : Experiences significant improvement with rerankers, with the best results from  CohereRerank  (0.876404 hit rate, 0.822753 MRR). llm-embedder : Benefits greatly from reranking, particularly with  CohereRerank  (0.882022 hit rate, 0.830243 MRR), which offers a substantial performance boost. Cohere : Cohere’s latest v3.0 embeddings outperform v2.0 and, with the integration of native CohereRerank, significantly improve its metrics, boasting a 0.88764 hit rate and a 0.836049 MRR. Voyage : Has strong initial performance that is further amplified by  CohereRerank  (0.91573 hit rate, 0.851217 MRR), suggesting high responsiveness to reranking. JinaAI : Very strong performance, sees notable gains with  bge-reranker-large  (0.938202 hit rate, 0.868539 MRR) and  CohereRerank  (0.932584 hit rate, 0.873689), indicating that reranking significantly boosts its performance. Google-PaLM : The model demonstrates strong performance, with measurable gains when using the  CohereRerank (0.910112

In [65]:
if LOG_TO_MLFLOW:
    for k, v in curated_mean_scores_df.T.to_dict(orient='records')[0].items():
        mlflow.log_metric(f"curated_response_eval__{k}", v)
    curated_deep_eval_df.to_html(f"{NOTEBOOK_CACHE_DP}/curated_deep_eval_df.html")
    mlflow.log_artifact(f"{NOTEBOOK_CACHE_DP}/curated_deep_eval_df.html", "curated_deep_eval_df")

# Clean up

In [66]:
if LOG_TO_MLFLOW:
    mlflow.end_run()

# Archive